### 1 - Import libs

**Execute only**

In [ ]:
import time
from datetime import datetime as dt
import matplotlib.pyplot as plt

import ee
import geemap
from sepal_ui.mapping import SepalMap

# initialize EE    
try:
    ee.Initialize()
except:
    ee.Authenticate()
    ee.Initialize()

import helpers as h

### 2 - Basic Input Variables

**DEFINE YOUR INPUT VARIABLES**

In [ ]:
# Define the projection to create the grid
# This should ideally be some equal area projection so that each sample represents the same area on the ground
crs = 'ESRI:54017' # Behrmanns equal area,
# see for others here https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3790990/#:~:text=The%20level%20of%20distortion%20can,compared%20to%20equal%2Darea%20maps.

# Area of Interest (can be both, a EE FeatureCollection, or a geopandas readable file format)
country = 'Kenya'   # country case
aoi = ee.FeatureCollection("FAO/GAUL/2015/level1").filter(ee.Filter.eq('ADM0_NAME', country))
#aoi = gpd.read_file('my_admin_boundaries.shp')

spacing = 100000 # in metres
sampling_strategy = 'systematic' # either 'random' or 'systematic'

#------------------------
# define an outputs 
#------------------------

# directory where output files will go
outdir = None # if left ot None, it goes into module results

# output a CEO compatible csv file
out_ceo_csv = True

# output a Geopackage file
out_gpkg = True

# ee asset
ee_asset_name = '01_myproject_sbae_points'

### 3 - Generate GRID and POINTS

### 3.1 - Systematic and random systematic grid

**Execute only**

In [ ]:
# Generate a systematic grid
grid_gdf, systematic_point_gdf = h.generate_samples(aoi, spacing, crs, 'systematic')

# Generate a random systematic grid
grid_gdf, random_point_gdf = h.generate_samples(aoi, spacing, crs, 'random')

### Plot grid and points

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(15, 15))
if isinstance(aoi, ee.FeatureCollection):
    geemap.ee_to_geopandas(aoi).to_crs(crs).plot(ax=ax, alpha=0.25)
else:
    aoi.to_crs(crs).plot(ax=ax, alpha=0.25)  
    
grid_gdf.plot(ax=ax, facecolor="none", edgecolor='black', lw=0.1)
systematic_point_gdf.plot(ax=ax, facecolor='red', markersize=.5)
random_point_gdf.plot(ax=ax, facecolor='green', markersize=.5)

#### Save points

In [ ]:
# upload to EE
h.upload_to_ee(systematic_point_gdf, ee_asset_name)

# save to local 
h.save_locally(systematic_point_gdf, ceo_csv=out_ceo_csv, gpkg=out_gpkg, outdir=outdir)

### 3.2 - Stratified Sampling

In [ ]:
# stratified sampling based on ESA LC
imageCollection = ee.ImageCollection("ESA/WorldCover/v100")
esaLc = imageCollection.filterBounds(aoi).first().clip(aoi)

stratifiedSamples = esaLc.stratifiedSample(**{
    "numPoints": 25,
    "region": aoi,
    "scale": 100, 
    "seed": 42, 
    "tileScale": 4, 
    "geometries": True
}).map(h.set_id)

### 4 - Display in map

**Execute only**

In [ ]:
# intialize Map
Map = SepalMap(['HYBRID'])
Map.zoom_ee_object(aoi.geometry())

# add layers
Map.addLayer(ee.Image().byte().paint(featureCollection=aoi, color=1, width=3), {"palette": "blue"}, 'Area of interest')
Map.addLayer(esaLc, False, 'ESA Worldcover 2020')
Map.addLayer(stratifiedSamples.draw(color="black", pointRadius=2), {}, 'Stratified Samples per class from ESA LC')

#
Map